In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# all_runs

- run locations

In [3]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
current run,..\..\..\..
2050 update (lower WFH and no telecommute),\\modelstation1\c$\workspace\sc_2050_base_year...
2050 update,\\modelstation1\c$\workspace\sc_2050_with_2023...
2050 RTP,L:\RTP_2022\final_runs\sc_rtp_2050_constrained...


## System Summary

In [4]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')

In [5]:
# Get light rail boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')

In [9]:
network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
#Light rail boardings
df_line_boardings = summary_data.load_agg_data('transit/transit_line_results.csv')
lr_boardings = df_line_boardings[df_line_boardings['mode']=='r']
lr_ridership = lr_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [10]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    'Light Rail Boardings': lr_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
source,,,,,,,
2050 RTP,"106,132,676","3,547,251","619,306","2,333,391","783,454",6.2%,"37,682"
2050 update,"97,332,812","3,066,138","356,300","1,338,012","394,811",4.4%,"34,533"
2050 update (lower WFH and no telecommute),"102,521,097","3,311,479","431,951","1,412,193","421,104",4.2%,"36,264"
current run,"82,983,139","2,557,051","239,416","485,465","92,876",2.1%,"41,738"


In [8]:
# # Populations near HCT and frequent transit
# df_time = pd.read_csv(r'../../../../outputs/agg/dash/trip_time_total.csv')
# df_ff = pd.read_csv(r'../../../../outputs/agg/dash/trip_sov_ff_time.csv')
# hh = pd.read_csv(r'../../../../outputs/agg/dash/hh_transit_dist.csv')
# person = pd.read_csv(r'../../../../outputs/agg/dash/person_transit_type.csv')

## Land Use

In [9]:
# Land Use Summary
df_parcels = summary_data.load_landuse('landuse/parcels_urbansim.txt')

In [10]:
# Summarize households and total jobs
df_pivot = pd.pivot_table(df_parcels, index='source', values=['hh_p','emptot_p'], aggfunc='sum')
df_pivot.rename(columns={'hh_p': 'Households', 'emptot_p':'Jobs'}, inplace=True)
# df_pivot = df_pivot.style.format('{:,.f}')
df_pivot[['Households','Jobs']].style.format('{:,.0f}')

,Households,Jobs
source,,
2050 RTP,"2,422,603","3,186,523"
2050 update,"2,421,059","3,161,923"
2050 update (lower WFH and no telecommute),"2,421,059","3,161,923"
current run,"1,736,129","2,190,946"


In [11]:
# Summarize jobs by sector
# df.columns

#### Jobs by Sector

In [12]:
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['empedu_p', 'empfoo_p', 'empgov_p', 'empind_p', 'empmed_p',
                                'empofc_p', 'empoth_p', 'empret_p', 'emprsc_p', 'empsvc_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'empedu_p': 'Education', 'empfoo_p':'Food', 'empgov_p':'Government',
                         'empind_p':'Industrial', 'empmed_p':'Medical', 'empofc_p':'Office',
                         'empoth_p':'Other', 'empret_p':'Retail', 'emprsc_p':'Resource', 'empsvc_p':'Service'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Education,Food,Government,Industrial,Medical,Office,Other,Retail,Resource,Service
source,,,,,,,,,,
2050 RTP,"282,038","269,830","220,584","367,503","358,855","924,772","162,475","379,308",0,"221,158"
2050 update,"287,539","270,522","179,539","369,405","359,833","923,858","163,738","386,158",0,"221,331"
2050 update (lower WFH and no telecommute),"287,539","270,522","179,539","369,405","359,833","923,858","163,738","386,158",0,"221,331"
current run,"167,143","157,285","222,537","341,587","242,463","582,273","155,306","176,184",0,"146,168"


#### Student Enrollment

In [13]:
# Students by grade level
df_pivot = pd.pivot_table(df_parcels, index='source', 
                          values=['stugrd_p', 'stuhgh_p', 'stuuni_p'], 
                           aggfunc='sum')
df_pivot.rename(columns={'stugrd_p': 'Grade School', 'stuhgh_p':'High School', 'stuuni_p':'College'}, inplace=True)
df_pivot.style.format('{:,.0f}')

,Grade School,High School,College
source,,,
2050 RTP,"517,319","206,601","224,710"
2050 update,"442,413","211,261","136,681"
2050 update (lower WFH and no telecommute),"442,413","211,261","136,681"
current run,"442,413","211,261","136,681"


In [14]:
# Join parcel lookup data to parcel files to do some county and district summaries
# Load parcel geography lookup from soundcast db
import sqlite3
conn = sqlite3.connect(r'../../../../inputs/db/soundcast_inputs_2023.db')
df_geog_lookup = pd.read_sql_query("SELECT ParcelID, CountyName, district_name FROM parcel_2023_geography", conn)

In [15]:
# Merge lookup data to parcels
df_parcels = df_parcels.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

#### Total Employment by Distict

In [16]:
# Group total employment by county and source
df_pivot = pd.pivot_table(df_parcels, index='CountyName', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,2050 RTP,2050 update,2050 update (lower WFH and no telecommute),current run
CountyName,,,,
King,"2,062,129","2,045,960","2,045,960","1,442,506"
Kitsap,"138,251","147,942","147,942","92,707"
Outside Region,549,0,0,0
Pierce,"517,554","473,614","473,614","334,336"
Snohomish,"468,040","494,407","494,407","321,397"


#### Total Employment by Distict

In [17]:
# Group total employment by district and source
df_pivot = pd.pivot_table(df_parcels, index='district_name', columns='source', values='emptot_p', aggfunc='sum')
df_pivot.style.format('{:,.0f}')

source,2050 RTP,2050 update,2050 update (lower WFH and no telecommute),current run
district_name,,,,
East Side,"693,197","615,476","615,476","420,109"
Everett-Lynwood-Edmonds,"125,550","295,998","295,998","158,328"
Kitsap,"138,252","147,942","147,942","92,707"
North Seattle-Shoreline,"296,871","190,173","190,173","147,826"
Renton-FedWay-Kent,"660,342","531,467","531,467","343,930"
S.Kitsap,"96,530","23,537","23,537","19,990"
Seattle CBD,"165,235","509,268","509,268","394,252"
South Pierce,"307,246","220,779","220,779","167,286"
Suburban Snohomish,"342,490","198,412","198,412","163,069"


## Synthetic Population

In [18]:
def count_by_county(output_path: str, summary_var: str, expfac_name: str, axis_name: str):
    df = summary_data.load_agg_data(output_path)

    tab = df.groupby(['source',summary_var])[expfac_name].sum().\
        unstack(summary_var).\
        rename_axis(columns={summary_var: axis_name})
    tab['Total'] = tab.sum(axis=1)

    display(tab.style.format('{:,.0f}'))

In [19]:
count_by_county('agg/dash/pptyp_county.csv','person_county','psexpfac','Population by county')

Population by county,King,Kitsap,Outside Region,Pierce,Snohomish,Total
source,,,,,,
2050 RTP,"3,029,767","353,461",nan,"1,213,634","1,230,341","5,827,203"
2050 update,"2,988,607","354,233",14,"1,237,744","1,207,563","5,788,161"
2050 update (lower WFH and no telecommute),"2,988,607","354,233",14,"1,237,744","1,207,563","5,788,161"
current run,"2,303,851","275,120",8,"925,555","851,485","4,356,019"


In [20]:
count_by_county('agg/dash/hh_geog.csv','hh_county','hhexpfac','Households by county')

Households by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
2050 RTP,"1,310,283","147,388","487,287","477,645","2,422,603"
2050 update,"1,184,801","60,986","308,856","298,332","1,852,975"
2050 update (lower WFH and no telecommute),"1,184,801","60,986","308,856","298,332","1,852,975"
current run,"866,673","39,114","196,913","187,282","1,289,982"


## employment by work location

In [21]:
count_by_county('agg/dash/person_worker_type.csv','person_work_county','psexpfac','Employment by county')

Employment by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
2050 RTP,"1,746,061","130,941","364,448","397,949","2,639,399"
2050 update,"1,660,249","119,626","403,512","406,486","2,589,873"
2050 update (lower WFH and no telecommute),"1,216,757","115,252","407,318","455,182","2,194,509"
current run,"1,273,680","84,057","305,156","285,123","1,948,016"


## Parking Costs

In [22]:
# Daily parking cost totals (ppricdyp)
pd.pivot_table(df_parcels, index='source', values='ppricdyp', aggfunc='sum').style.format('{:,.0f}')

,ppricdyp
source,
2050 RTP,"100,944,431"
2050 update,"102,112,889"
2050 update (lower WFH and no telecommute),"102,112,889"
current run,"1,029,600"


In [23]:
# # Parking costs by district
# pd.pivot_table(df_parcels, index='district_name', columns='source', values='pprichrp', aggfunc='sum').style.format('{:,.0f}')

In [24]:
# df_buffered.columns

In [25]:
# # Check buffered parcels distance to light rail
# df_buffered = summary_data.load_landuse('landuse/buffered_parcels.txt')

# # Merge geography to file
# df_buffered = df_buffered.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

In [26]:

# df_buffered.groupby(['source','district_name'])['dist_lbus'].describe()
# # df_buffered.columns